<a href="https://colab.research.google.com/github/Batovs/MAP2110---Projeto-6---Caos/blob/main/MAP2212_EP5_vers%C3%A3o_limpa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Bibliotecas
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma

In [145]:
#INFO da dupla
INFO = {5381328:"Nikolas Lukin",11811012:"Vinícius Collaço"}

class Estimador:
  """
  Classe para criar o objeto, ele recebe valores para os vetores x e y.
  Os metodos definidos abaixo serao utilizadas por um corretor automatico. 
  Portanto, precisa manter os outputs e inputs dos 2 metodos abaixo. 
  """
  def __init__(self,x,y):
    """
    Inicializador do objeto. Este metodo recebe 
    valores pros vetores x e y em formato de lista 
    e implementa no objeto. 
    """
    
    self.vetor_x = np.array(x) #formato: [0,0,0] - List cujo len(x) = 3
    self.vetor_y = np.array(y) #formato: [0,0,0] - List cujo len(y) = 3

    self.alpha = self.vetor_x + self.vetor_y
    self.n = 3900000 #15366400 #Definido no relatorio
    self.queima = 1000 #1000
    self.k = 2000 #Numero de bins definido no relatorio

    #calcula a constante de normalização
    beta = (np.prod(gamma(self.alpha))) / (gamma(sum(self.alpha)))
    self.const_norm = 1/beta

    self.covariancia = self.matriz_de_covariancia()
    self.M_cov_reduzida = self.covariancia[:2, :2]

    self.potenciais = self.amostra_MCMC(self.vetor_x, self.vetor_y, self.n)
    self.f_ordenada = self.f_norm_ord()
    self.f_min = self.f_ordenada[0]
    self.f_max = self.f_ordenada[-1]
    self.f_bins = self.f_bins()

  def matriz_de_covariancia(self):
    """
    Funcao que gera a matriz de covariancia sigma, 
    com base na variancia e covariancia da 
    https://en.wikipedia.org/wiki/Dirichlet_distribution
    """
    size = len(self.alpha)
    M_sigma = np.identity(size)
    a_0 = sum(self.alpha)
      
    for i in range(size):
      for j in range(size):
        if i == j:#Variancia da Dirichilet            
          M_sigma[i][j] = (a_0 - self.alpha[i])*self.alpha[i]/((a_0**2)*(a_0 + 1))            
        else:#Covariancia da Dirichilet
          M_sigma[i][j] = -1*self.alpha[i]*self.alpha[j]/((a_0**2)*(a_0 + 1))   
    
    return M_sigma

  def gera_candidato(self, atual):
    """metodo para gerar um ponto candidato no simplex"""
    while True:
      passo = np.random.multivariate_normal([0, 0], self.M_cov_reduzida)
      candidato = ([atual[0] + passo[0],
                    atual[1] + passo[1],
                    1 - (atual[0] + passo[0] + atual[1] + passo[1])]
                  )
      if all(i > 0 for i in candidato) > 0:
        return candidato
  
  def f_theta(self,theta):
    """metodo para calcular o potencial de f(theta)"""
    f = np.prod(np.power(theta, self.alpha - 1))
    return f
  
  def amostra_MCMC(self,x,y,n):
    """
    Funcao que recebe valores pros vetores x e y, o tamanho n da amostra, 
    gera uma amostra de tamanho n a partir do metodo de monte carlo via 
    cadeias de markov, onde cada elemento da amostra tem tamanho 3 (vetor),
    e retorna uma lista de tamanho n com os potenciais de cada ponto obtido,
    onde cada elemento tem tamanho 1 (escalar).
    
    Nao utilize a fuancao densidade de probabilidade, apenas a funcao potencial!
    """
    amostras = [[1/3, 1/3, 1/3]] #ponto inicial, centro do simplex
    alpha = np.array(x) + np.array(y)

    for _ in range(n + self.queima):
      candidato = self.gera_candidato(amostras[-1])

      #probabilidade de aceitacao
      prob = min(1, self.f_theta(candidato) / self.f_theta(amostras[-1]))

      #aceita com a probabilidade calculada
      if np.random.random() < prob:
        amostras.append(candidato)     
      
      amostras.append(amostras[-1])

    amostra_de_potenciais = np.prod(np.power(np.array(amostras), alpha - 1), axis=1)
    return amostra_de_potenciais[self.queima+1:] # Exemplo do formato = [0.04867, 0.00236, 0.00014 ... ]

  def f_norm_ord(self):
    """metodo para normalizar e ordenar a potencial gerada da cadeia de Markov"""
    f = self.potenciais
    f.sort()
    f_normalizada = f*self.const_norm

    return f_normalizada
  
  def f_bins(self):
    """metodo para separar a f em bins"""

    f_normalizada = self.f_ordenada
    f_bins = [0]*self.k

    #Separa em bins com uma quantidade constante de pontos cada bin
    passo = int(self.n/self.k)

    for i in range (0,self.k):
        f_bins[i] = f_normalizada[i*passo]

    return f_bins
  
  def U(self,v):
    """
    Este metodo recebe um valor para v e, a partir dele, retorna U(v|x,y) 
    a partir dos vetores x e y inicializados anteriormente
    """
    f_ordenado = self.f_bins

    if v > self.f_max:
      return 1
    if v < self.f_min:
      return 0  

    # numero de bins abaixo de um certo v
    menor_que_v = np.searchsorted(f_ordenado, v, side='left')
    u = menor_que_v/self.k

    return u    

In [142]:
x = [4, 6, 4]
y = [1, 2 ,3]
estimador = Estimador(x, y)

In [72]:
cand = estimador.gera_candidato([0.3,0.3,0.4])
estimador.f_theta(cand)/estimador.f_theta([0.3,0.3,0.4])

1.5081890504638509

In [143]:
estimador.f_bins

[0.019024965563938307,
 0.01949283935057085,
 0.03299033927379696,
 0.045037641648076596,
 0.046460906696114954,
 0.05328965198530543,
 0.06003999941897669,
 0.08286862366126475,
 0.08571594688708577,
 0.0920084739280938,
 0.0983424222226587,
 0.13679321086309015,
 0.1400940846135284,
 0.15801446155279614,
 0.16568860963632817,
 0.16602184410274615,
 0.19355941069001742,
 0.20095861176166382,
 0.20774928262871534,
 0.21789310246530513,
 0.21816754069781893,
 0.22882866459302054,
 0.230808857652865,
 0.2313673673862851,
 0.23963030742992222,
 0.2501594704280859,
 0.26516028536870784,
 0.26972599974897743,
 0.2752695933825485,
 0.27906421243484325,
 0.29010878095166537,
 0.2918478427957205,
 0.29807516812103274,
 0.300152037687349,
 0.30735626977075636,
 0.3116616881162434,
 0.31514580818835625,
 0.3282454484537416,
 0.3311706149557366,
 0.3359370333467314,
 0.3541171116190025,
 0.35698850259021175,
 0.36714224982318705,
 0.37231454100163835,
 0.37269605938289757,
 0.3760351645179736,
 0

In [136]:
estimador.f_min

0.007144073284347175

In [137]:
estimador.f_max

16.59705304310065

In [65]:
estimador.const_norm

1396755360.0

In [139]:
estimador.amostras

[[0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
 [0.3333333333333333, 0.3333333333333333, 0.3333333333333333],
 [0.3352288670854523, 0.41831082291537675, 0.2464603099991709],
 [0.3352288670854523, 0.41831082291537675, 0.2464603099991709],
 [0.2904971354043342, 0.4927327295557742, 0.21677013503989162],
 [0.2904971354043342, 0.4927327295557742, 0.21677013503989162],
 [0.2904971354043342, 0.4927327295557742, 0.21677013503989162],
 [0.2284382326207, 0.5884784885766633, 0.18308327880263675],
 [0.2284382326207, 0.5884784885766633, 0.18308327880263675],
 [0.19929050294973888, 0.5248941611113028, 0.27581533593895835],
 [0.19929050294973888, 0.5248941611113028, 0.27581533593895835],
 [0.1643941447466961, 0.5004425526663533, 0.3351633025869506],
 [0.1643941447466961, 0.5004425526663533, 0.3351633025869506],
 [0.1643941447466961, 0.5004425526663533, 0.3351633025869506],
 [0.22268635736403938, 0.5155508143175535, 0.26176282831840714],
 [0.22268635736403938, 0.5155508143175535, 0.261

In [132]:
estimador.gera_candidato([-1,0.5,1.5])

[-0.9361713254370239, 0.6300491520570028, 1.306122173380021]

In [133]:
estimador.M_cov_reduzida

array([[ 0.00892857, -0.0047619 ],
       [-0.0047619 ,  0.01142857]])